In [1]:
import tensorflow as tf
import utils_img_rec as ut
import pickle
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
import pathlib

### Definindo variaveis

In [2]:
DATA_DIR  = '../bases/pickle/imagens/mnist/mnist-train-pickle.pickle'
TEST2_DIR = '../bases/dados/imagens/teste2/amostra-numerica'
formato = 'jpg'
IMG_SIZE = 28

DATA_DIR = pathlib.Path(DATA_DIR)
TEST2_DIR = pathlib.Path(TEST2_DIR)

pickle_in = open(DATA_DIR,"rb")
data_train = pickle.load(pickle_in)

### Definindo categorias

In [3]:
#CATEGORIES = []
CATEGORIES = ut.get_classes(data_train)
CATEGORIES.sort()
try:
    CATEGORIES.remove('.ipynb_checkpoints')
except:
    pass
print(CATEGORIES)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


### Carregando o modelo

In [4]:
model = tf.keras.models.load_model('../modelos_salvos/tensorflow/modelo.h5')

In [5]:
def prepare(filepath):
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array, new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

def plot_image(prediction_array, true_label, img):
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img, cmap='gray')
    predicted_label = np.argmax(prediction_array)        
    if predicted_label == true_label:
        color = 'green'
    else:
        color = 'red'
    plt.xlabel("Classe - {} | {:2.0f}% (true class {})".format(CATEGORIES[predicted_label], 100*np.max(prediction_array), CATEGORIES[true_label]), color=color)

def plot_value_array(prediction_array, true_label):
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    thisplot = plt.bar(range(len(CATEGORIES)), prediction_array, color= "#777777")
    plt.ylim([0, 1])
    predicted_label = np.argmax(prediction_array)

    thisplot[predicted_label].set_color('red')
    print(CATEGORIES[true_label])
    thisplot[true_label].set_color('green')

In [6]:
for category in CATEGORIES:

    true_label_num = CATEGORIES.index (category)
    
    DIR = TEST2_DIR + '/previsao ' + '(' + category + ').' + formato

    xp, Xp = prepare(DIR)
    Xp=np.array(Xp/255.0)
    prediction = model.predict( Xp )
    prev = prediction[0]

    plt.figure(figsize=(6,3))
    plt.subplot(1,2,1)

    plot_image(prediction[0], true_label_num, xp)
    plt.subplot(1,2,2)
    plot_value_array(prediction[0],  true_label_num)
    plt.show()

    for i in range( len(prev) ):
        print( 'Classe - {} | Confiança - {} %'.format(CATEGORIES[i], np.round(prev[i]*100, 2) ) )

    inp = model.inputs 
    
    k=0
    for layer in model.layers:
        if 'conv' in layer.name: 
            fig= plt.figure(figsize=(5,5))
            out = layer.output
            feature_map = Model(inputs= inp, outputs= out)
            f = feature_map.predict(Xp)
              
            feature_map.summary()
            
            print('Ativação na {}° camada'.format(k+1))

            for i in range(f.shape[3]):

                p = f[0,:,:,i]

                ax = fig.add_subplot(8,8,i+1)
                ax.imshow(p, cmap='gray')

            plt.show()
        k=k+1
    print('\n\n\n')

TypeError: unsupported operand type(s) for +: 'PosixPath' and 'str'